In [ ]:
#DrugBank_crawling
import requests
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings
from lxml import etree
import openpyxl
import pandas as pd

wb = openpyxl.Workbook()
sheet1 = wb.active
i, j = 0, 1
sheet1.append(['Drugbank', 'Name', 'sysnomy', 'Molecular Formular', 'Molecular Weight', 'svg', '2D', 'IUPAC Name', 'Inchi', 'Isomeric Smile', 'Polar sur Area',
               'Hydrogen DC', 'Hydrogen AC', 'Rotatable BC', 'Action',
               'physioloical charge', 'physical Descrption', 'Melting P', 'Hydrophobicity', 'Solubility', 'LogP',
               'Mass Spect', 'Biological Activity','Binding target',
               'Drug indication', 'Associatie Con', 'pharamcology', 'Absorption', 'V of Distru',
               'P Binding', 'Route of E', 'Metabolism',
               'Biological Halflife', 'Clearance', 'Toxicity', 'Pathway', 'PubmedID', 'summary', 'Background', 'Groups',
               'Compound'])
sheet2 = wb.create_sheet('title')
sheet2.title = 'new sheet'
sheet1.sheet_properties.tabColor = '000000'
disable_warnings(InsecureRequestWarning)
if __name__ == '__main__':
    headers = {
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.54"
    }
    url_main = 'https://go.drugbank.com/drugs/'
    # keyword = pd.read_excel('fileName.xlsx', dtype=object).loc[:, 'Column_Name']
    keyword = [ 'DB02414', 'DB04433', 'DB04350', 'DB03632', 'DB13798', 'DB00104', 'DB00063']# Test data
    
    for i in range(len(keyword)):
        url = url_main + keyword[i]
        print(url)
        DI_li = []
        name, synonmy, MF, MW, svg, TowD, IUPACName, Inchi, IsomericS, PSA, HDC, HAC, RBC, PC = '', '', '', '', '', '', '', '', '', '', '', '', '', ''
        PD, MP, Hydrop, Solu, LogP, MassSpect, BA, TO, BT, function, DI, AssCon, Phara, Action ='', '', '', '', '', '', '', '', '', '', '', '', '', ''
        Abosrp, VofD, PB, RofE, Metab, BH, Clear, Toxi, Path, PID, Sum, Bcak, Groups, Compound = '', '', '', '', '', '', '', '', '', '', '', '', '', ''
        response = requests.get(url=url, headers=headers, verify=False)
        response.encoding = response.apparent_encoding
        tree = etree.HTML(response.text)
        name = tree.xpath('//dt[@id="generic-name"]/following-sibling::dd[1]/text()')
        name = name[0]  if name else ''
        li_synonmy = tree.xpath('//dt[@id="synonyms"]/following-sibling::dd[1]/ul/li')
        if li_synonmy:
            for li in li_synonmy:
                synonmy = li.xpath('./text()')[0] + ';' + synonmy
        MW = ''.join(tree.xpath('//dt[@id="weight"]/following-sibling::dd[1]//text()'))
        MF = ''.join(tree.xpath('//dt[@id="chemical-formula"]/following-sibling::dd[1]//text()'))
        svg_url = tree.xpath('//div[@class="structure"]/a/@href')
        if svg_url:
            print('https://go.drugbank.com/'+svg_url[0])
            svg = keyword[i] + '.svg'   
        TowD_list = tree.xpath('//div[@aria-labelledby="structure-download"]/a')
        if TowD_list:
            for a in TowD_list:
                TowD_name = a.xpath('./text()')
                if TowD_name[0] == 'PDB':
                    TowD = keyword[i] + '.pdb'
                    TowD_url = a.xpath('./@href')
                    print('https://go.drugbank.com/'+TowD_url[0])  
        IUPACName = ''.join(tree.xpath('//dt[@id="iupac-name"]/following-sibling::dd[1]/div/text()'))
        Inchi = ''.join(tree.xpath('//dt[@id="inchi"]/following-sibling::dd[1]/div/text()'))
        IsomericS = ''.join(tree.xpath('//dt[@id="smiles"]/following-sibling::dd[1]/div/text()'))
        drug_moldb = tree.xpath('//*[@id="drug-moldb-properties"]/tbody/tr')
        for tr in drug_moldb:
            td_mane = tr.xpath('./td[1]/text()')
            PC = tr.xpath('./td[2]//text()')[0]  if td_mane[0] == 'Physiological Charge' else ''
            HAC = tr.xpath('./td[2]//text()')[0]  if td_mane[0] == 'Hydrogen Acceptor Count' else ''
            HDC = tr.xpath('./td[2]//text()')[0]  if td_mane[0] == 'Hydrogen Donor Count' else ''
            PSA = tr.xpath('./td[2]//text()')[0]  if td_mane[0] == 'Polar Surface Area' else ''
            print(PSA.strip())
            RBC = tr.xpath('./td[2]//text()')[0]  if td_mane[0] == 'Rotatable Bond Count' else ''
            if td_mane[0] == 'logP':
                LogP = tr.xpath('./td[2]//text()')[0] + '(' + tr.xpath('./td[3]//text()')[0] + ');' + LogP
            Solu = tr.xpath('./td[2]//text()')[0]  if td_mane[0] == 'Water Solubility' else ''
        PD = ''.join(tree.xpath('//div[@id="state"]/following-sibling::dd[1]/text()'))
        experl_proper = tree.xpath('//table[@id="experimental-properties"]/tbody/tr')
        for tr in experl_proper:
            td_mane = tr.xpath('./td[1]/text()')
            Hydrop = tr.xpath('./td[2]//text()')[0]  if td_mane[0] == 'hydrophobicity' else ''
            MP = tr.xpath('./td[2]//text()')[0]  if td_mane[0] == 'melting point (°C)' else ''
        MassSpect = ''.join(tree.xpath('//div[@id="mass-spec"]/following-sibling::dd[1]//text()'))
        Action = ''.join(tree.xpath('//dt[@id="mechanism-of-action"]/following-sibling::dd[1]/p/text()'))
        BT_list = tree.xpath('//table[@id="drug-moa-target-table"]/tbody/tr')
        if BT_list:
            for tr in BT_list:
                TO = tr.xpath('./td[3]/text()')
                TO = '[' + TO[0] + ']'  if TO else ''
                function = tr.xpath('./td[2]/div/text()')
                function = '-' + function[0]  if function else ''
                BT = tr.xpath('./td[1]/a/text()')[0] + TO + function + ';' + BT
        DI = ''.join(tree.xpath('//dt[@id="indication"]/following-sibling::dd[1]/p/text()'))
        AssCon_list = tree.xpath('//dt[@id="associated-conditions"]/following-sibling::dd[1]/ul/li')
        if AssCon_list:
            for li in AssCon_list:
                AssCon = li.xpath('.//text()')[0] + ';' + AssCon
        Phara = ''.join(tree.xpath('//dt[@id="pharmacodynamics"]/following-sibling::dd[1]/p/text()'))
        Abosrp = ''.join(tree.xpath('//dt[@id="absorption"]/following-sibling::dd[1]/p/text()'))
        VofD = ''.join(tree.xpath('//dt[@id="volume-of-distribution"]/following-sibling::dd[1]/p/text()'))
        PB = ''.join(tree.xpath('//dt[@id="protein-binding"]/following-sibling::dd[1]/p/text()'))
        RofE = ''.join(tree.xpath('//dt[@id="route-of-elimination"]/following-sibling::dd[1]/p/text()'))
        Metab = ''.join(tree.xpath('//dt[@id="metabolism"]/following-sibling::dd[1]/p/text()'))
        BH = ''.join(tree.xpath('//dt[@id="half-life"]/following-sibling::dd[1]/p/text()'))
        Clear = ''.join(tree.xpath('//dt[@id="clearance"]/following-sibling::dd[1]/p/text()'))
        Toxi = ''.join(tree.xpath('//dt[@id="toxicity"]/following-sibling::dd[1]/p/text()'))
        Path = ''.join(tree.xpath('//dt[@id="pathways"]/following-sibling::dd[1]/p/text()'))
        cite_list = tree.xpath('//dt[@id="general-references"]/following-sibling::dd[1]/ol/li')
        if cite_list:
            for li in cite_list:
                AT_mane = li.xpath('./a/text()')
                if AT_mane[0] == 'Article':
                    PID_url = 'https://go.drugbank.com/' + li.xpath('./a/@href')[0]
                    PID_res = requests.get(url=PID_url, headers=headers, verify=False)
                    PID_res.encoding = PID_res.apparent_encoding
                    PID_tree = etree.HTML(PID_res.text)
                    PID = PID_tree.xpath('//dt[@id="pubmed-id"]/following-sibling::dd[1]/div/text()')[0].replace('[', '') + ';' + PID
        Sum = ''.join(tree.xpath('//dt[@id="summary"]/following-sibling::dd[1]/p/text()'))
        Bcak = ''.join(tree.xpath('//dt[@id="background"]/following-sibling::dd[1]/p/text()'))
        Groups = ''.join(tree.xpath('//dt[@id="groups"]/following-sibling::dd[1]//text()'))
        Compound = ''.join(tree.xpath('//dt[@id="pubchem-compound"]/following-sibling::dd[1]/a/text()'))
        xx_info = [keyword[i], name, synonmy, MF, MW, svg, TowD, IUPACName, Inchi, IsomericS, PSA, HDC, HAC, RBC, Action, PC, PD, MP, Hydrop, Solu,
                   LogP, MassSpect, BA, BT, DI, AssCon, Phara, Abosrp, VofD, PB, RofE, Metab, BH, Clear, Toxi, Path, PID, Sum, Bcak, Groups, Compound]
        sheet1.append(xx_info)
    wb.save('drugbank-info.xlsx')
